In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
DF = pd.read_csv("../../datasets/lane_closure.csv",low_memory=False)
DF.head()

,Primary Street,Cross Street,Boundaries,Direction,Date Closed - From,Date Closed - To,Traffic Effect,Organization,Time Closed - From,Time Closed - To,...,Inserted Date,Geometry Id,Status,Complete Closure,KML,Latitude,Longitude,X,Y,Geometry
0,Bryce St,River Av,River Av to End,Eastbound & Westbound,August 16 2021,October 16 2021,"Eastbound curb lane, closed for building const...",Zelko,12:00 AM,12:00 AM,...,08/11/2021 06:37:43 PM,552963,Current,No,<LineString><extrude>0</extrude><tessellate>0<...,49.879699,-97.142781,633429.365245,5.526909e+06,MULTILINESTRING ((-97.142387643857 49.87915381...
1,Lilac St,Carter Av,Carter Av to Weatherdon Av,Southbound,June 14 2021,October 29 2021,"East side sidewalk, closed for building constr...",Globeland Construction Ltd,12:00 AM,12:00 AM,...,06/09/2021 10:35:09 AM,550863,Current,No,<LineString><extrude>0</extrude><tessellate>0<...,49.862454,-97.151445,632854.263781,5.524977e+06,MULTILINESTRING ((-97.151173092203 49.86211919...
2,Princess St,Pacific Av,Pacific Av to Rupert Av,Southbound,August 14 2020,October 16 2021,"West side sidewalk, covered by pedestrian scaf...",Hofer Const. Ltd.,12:00 AM,12:00 AM,...,05/21/2021 12:12:59 PM,550250,Current,No,<LineString><extrude>0</extrude><tessellate>0<...,49.901786,-97.139766,633584.982267,5.529370e+06,MULTILINESTRING ((-97.139915854251 49.90158994...
3,Henry Av,Austin St,Austin St to End,Westbound,May 10 2021,November 30 2021,"Westbound right turnaround lane, closed for st...",Bockstael Construction Limited,12:00 AM,11:00 PM,...,05/20/2021 09:32:11 AM,550156,Current,No,<LineString><extrude>0</extrude><tessellate>0<...,49.903340,-97.134152,633983.759759,5.529553e+06,MULTILINESTRING ((-97.13466284795 49.903503694...
4,Grandin St,Tache Av,Tache Av to St Joseph St,Eastbound,May 31 2021,January 13 2022,"South side sidewalk, closed for building const...",Qualico,12:00 AM,12:00 AM,...,05/26/2021 03:38:27 PM,550361,Current,No,<LineString><extrude>0</extrude><tessellate>0<...,49.896013,-97.126315,634566.885434,5.528752e+06,MULTILINESTRING ((-97.127616387474 49.89567885...


In [3]:
for col in DF.columns:
    print(col,DF[col].dtype)

Primary Street object
Cross Street object
Boundaries object
Direction object
Date Closed - From object
Date Closed - To object
Traffic Effect object
Organization object
Time Closed - From object
Time Closed - To object
Lane Closure ID int64
Modified Date object
Inserted Date object
Geometry Id int64
Status object
Complete Closure object
KML object
Latitude float64
Longitude float64
X float64
Y float64
Geometry object


In [33]:
DF = pd.read_csv("../../datasets/lane_closure.csv",low_memory=False)
DF = DF.rename(columns={"Primary Street":"Street","Cross Street":"At","Latitude":"Lat","Longitude":"Long"})
DF.loc[:,"Street"] = [" ".join(l[:-1]) for l in DF["Street"].str.split()]
DF.loc[:,"At"] = [" ".join(l[:-1]) for l in DF["At"].str.split()]
DF.loc[:,"Boundaries"] = DF["Boundaries"].str.split(" to ")
DF.loc[:,"Direction"] = DF["Direction"].str.split(" & ")
__clean_string = lambda l: [[[(float(j.split()[1]),float(j.split()[0])) for j in s.split(", ")] for s in m] for m in l.values]
DF.loc[:,"Geometry"] = __clean_string(DF["Geometry"].str.replace("MULTILINESTRING \(\(|\)\)","",regex=True).replace().str.split("\), \("))

def _num_points(m):
    set_i = set()
    for l in m: set_i |= set(l)
    return set_i 
DF["Num_Segments"] = [len(i) for i in DF["Geometry"].values]
DF["Num_Points"] = [len(_num_points(i)) for i in DF["Geometry"].values]

DF = DF.drop(columns=["Traffic Effect","Organization","Lane Closure ID","Modified Date","Inserted Date","Geometry Id","Status","KML","X","Y"])
DF

/Users/jasetran/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3162: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).ndim


,Street,At,Boundaries,Direction,Date Closed - From,Date Closed - To,Time Closed - From,Time Closed - To,Complete Closure,Lat,Long,Geometry,Num_Segments,Num_Points
0,Bryce,River,"[River Av, End]","[Eastbound, Westbound]",August 16 2021,October 16 2021,12:00 AM,12:00 AM,No,49.879699,-97.142781,"[[(49.879153816388, -97.142387643857), (49.880...",1,3
1,Lilac,Carter,"[Carter Av, Weatherdon Av]",[Southbound],June 14 2021,October 29 2021,12:00 AM,12:00 AM,No,49.862454,-97.151445,"[[(49.862119196838, -97.151173092203), (49.862...",1,5
2,Princess,Pacific,"[Pacific Av, Rupert Av]",[Southbound],August 14 2020,October 16 2021,12:00 AM,12:00 AM,No,49.901786,-97.139766,"[[(49.90158994935, -97.139915854251), (49.9019...",1,3
3,Henry,Austin,"[Austin St, End]",[Westbound],May 10 2021,November 30 2021,12:00 AM,11:00 PM,No,49.903340,-97.134152,"[[(49.903503694415, -97.13466284795), (49.9033...",1,4
4,Grandin,Tache,"[Tache Av, St Joseph St]",[Eastbound],May 31 2021,January 13 2022,12:00 AM,12:00 AM,No,49.896013,-97.126315,"[[(49.895678857718, -97.127616387474), (49.896...",1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
425,Sherbrook,Logan,"[Logan Av, William Av]",[Southbound],September 20 2021,November 01 2021,12:00 AM,12:00 AM,No,49.906584,-97.153705,"[[(49.904179261579, -97.155653416614), (49.904...",1,30
426,Gull Lake,Markham,"[Markham Rd, East Lake Dr]","[Northbound, Southbound]",September 17 2021,October 08 2021,12:00 AM,12:00 AM,No,49.809311,-97.158976,"[[(49.80730904568, -97.158914003228), (49.8081...",1,39
427,Wardlaw,Osborne,"[Osborne St, Scott St]",[Eastbound],September 20 2021,October 15 2021,12:00 AM,12:00 AM,No,49.877534,-97.142232,"[[(49.876961712509, -97.143938755153), (49.877...",1,5
428,Scotsborough,Beckinsale,"[Beckinsale By, Novavista Dr]",[Southbound],September 23 2021,October 13 2021,12:00 AM,12:00 AM,No,49.820905,-97.107388,"[[(49.820524519173, -97.107184511392), (49.820...",1,4


In [34]:
DF.to_csv("clean_datasets/LANE_CLOSURE.csv")

In [32]:
def _num_points(m):
    set_i = set()
    for l in m: set_i |= set(l)
    return set_i 
df["Num_Segments"] = [len(i) for i in DF["Geometry"].values]
df["Num_Points"] = [len(_num_points(i)) for i in DF["Geometry"].values]
df

,Geometry,Num_Segments,Num_Points
0,"[[(49.879153816388, -97.142387643857), (49.880...",1,3
1,"[[(49.862119196838, -97.151173092203), (49.862...",1,5
2,"[[(49.90158994935, -97.139915854251), (49.9019...",1,3
3,"[[(49.903503694415, -97.13466284795), (49.9033...",1,4
4,"[[(49.895678857718, -97.127616387474), (49.896...",1,2
...,...,...,...
425,"[[(49.904179261579, -97.155653416614), (49.904...",1,30
426,"[[(49.80730904568, -97.158914003228), (49.8081...",1,39
427,"[[(49.876961712509, -97.143938755153), (49.877...",1,5
428,"[[(49.820524519173, -97.107184511392), (49.820...",1,4


In [31]:
m = DF["Geometry"].values[0]
def _num_points(m):
    set_i = set()
    for l in m: set_i |= set(l)
    return set_i 
len(_num_points(DF["Geometry"].values[411]))

243

In [20]:
df.iloc[425]

Geometry        [[(49.904179261579, -97.155653416614), (49.904...
Num_Segments                                                    1
Num_Points                                                     60
Name: 425, dtype: object

In [16]:
[len(_num_points(i)) for i in DF["Geometry"].values]

<ipython-input-16-e26db84c41e8>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  [np.array(i).ravel() for i in DF["Geometry"].values]


[array([ 49.87915382, -97.14238764,  49.88013648, -97.14317808,
         49.88024478, -97.14315901]),
 array([ 49.8621192 , -97.15117309,  49.8624542 , -97.15144463,
         49.86266724, -97.15161732,  49.86277601, -97.15170549,
         49.86278936, -97.15171632]),
 array([ 49.90158995, -97.13991585,  49.9019604 , -97.13963211,
         49.90198238, -97.13961527]),
 array([ 49.90350369, -97.13466285,  49.90337458, -97.13425606,
         49.90327065, -97.13392862,  49.90317638, -97.13364139]),
 array([ 49.89567886, -97.12761639,  49.8963466 , -97.12501348]),
 array([ 49.88975135, -97.14578837,  49.88983667, -97.14583476,
         49.89113601, -97.14654041,  49.89122135, -97.14658676]),
 array([ 49.94879747, -97.04786261,  49.94851419, -97.04722547]),
 array([ 49.88406787, -97.16829316,  49.88403706, -97.16713629]),
 array([ 49.96094868, -97.08725191,  49.96152177, -97.08626457,
         49.96230657, -97.08491733,  49.96254032, -97.08451449]),
 array([ 49.86819328, -97.16418362,  49.86